# Training program for Experiment

## setting path

In [1]:
import sys
import os

sys.path.append(os.path.split(os.getcwd())[0])


## データプレプロセス

## vocablary作成
すでにvocab.txtが作成済みである場合不用

In [2]:
from fast_jtnn.mol_tree import MolTree
from getpass import getpass,getuser

import mysql
from mysql import connector
import warnings

# SQL server profile
host = "localhost"
user = None
passwd = None
port = 3306
database="chemoinfo"

# 
VOCAB_FILE = "./MS_vocab.txt"

# get massbank data from SQL server
try:
    if not isinstance(user,str):
        user = raw_input("user")
    if not isinstance(passwd,str):
        passwd = getpass()
    connect = connector.connect(host=host,user=user,password=passwd,port=port,database=database)
    cursor = connect.cursor()
    cursor.execute("""select smiles from massbank where ms_type="MS" and instrument_type="EI-B" and smiles<>'N/A'; """)
    smiles_list = cursor.fetchall()
except mysql.connector.Error as e:
    print("Something went wrong: {}".format(e))
    sys.exit(1)
finally:
    if passwd : del passwd
    if connect: connect.close()
    if cursor: cursor.close()

# create vocablary
succes = 0
fault = 0
cset = set()
for one in smiles_list:
    try:
        mol = MolTree(one[0])
    except AttributeError as e:
        warnings.warn("Entered An SMILES that does not meet the rules")
        continue
    for c in mol.nodes:
        cset.add(c.smiles)

# write vocab
with open(VOCAB_FILE,"w") as f:
    for one in cset:
        f.write(one+"\n")

user a
 ·


Something went wrong: 1045 (28000): Access denied for user 'a'@'localhost' (using password: YES)


NameError: name 'connect' is not defined

! mkdir vae_model/
%run ../fast_molvae/vae_train.py --train processed --vocab ./MS_vocab.txt --save_dir vae_model/

# Vocab,datasetのロード

In [2]:
from fast_jtnn import *
from MS_PredictModel import MS_Dataset,MS_Dataset_pickle,dataset_load
import pickle
import torch

VOCAB_FILE = "./MS_vocab.txt"

vocab = [x.strip("\r\n ") for x in open(VOCAB_FILE,"r")]
vocab = Vocab(vocab)

'''
MS_Dataset.QUERY = """select smiles,file_path from massbank where ms_type="MS" and instrument_type="EI-B" and smiles<>'N/A';"""
dataset = MS_Dataset(vocab=vocab,host="localhost",database="chemoinfo",batch_size=20)
'''
train_vali_rate = 0.9

train_dataset, vali_dataset = dataset_load("./massbank.pkl",vocab,20,train_vali_rate)
print("number of train dataset :",len(train_dataset))
print("number of validation dataset :",len(vali_dataset))

('number of train dataset :', 6716)
('number of validation dataset :', 747)


## モデルの作成

In [3]:
from ms_encoder import ms_peak_encoder,ms_peak_encoder_lstm
import torch.nn as nn
import torch
hidden_size = 100
latent_size = 56
depthT = 20
depthG = 3

dec_model = JTNNVAE(vocab, hidden_size, latent_size, depthT, depthG).to('cuda')
print dec_model
enc_model = ms_peak_encoder_lstm(train_dataset.max_spectrum_size,output_size=latent_size,hidden_size=200,embedding_size=5,num_rnn_layers=1,bidirectional=False).to('cuda')
print enc_model

for param in dec_model.parameters():
    if param.dim() == 1:
        nn.init.constant_(param, 0)
    else:
        nn.init.xavier_normal_(param)
load_model = "./vae_model/model.iter-110000"
dec_model.load_state_dict(torch.load(load_model,map_location='cuda'))

print "Model #Params: %dK" % (sum([x.nelement() for x in dec_model.parameters()]) / 1000,)
print "Model #Params: %dK" % (sum([x.nelement() for x in enc_model.parameters()]) / 1000,)

/usr/local/anaconda-aisiars/envs/jtvae/lib/python2.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


JTNNVAE(
  (jtnn): JTNNEncoder(
    (embedding): Embedding(365, 100)
    (outputNN): Sequential(
      (0): Linear(in_features=200, out_features=100, bias=True)
      (1): ReLU()
    )
    (GRU): GraphGRU(
      (W_z): Linear(in_features=200, out_features=100, bias=True)
      (W_r): Linear(in_features=100, out_features=100, bias=False)
      (U_r): Linear(in_features=100, out_features=100, bias=True)
      (W_h): Linear(in_features=200, out_features=100, bias=True)
    )
  )
  (decoder): JTNNDecoder(
    (embedding): Embedding(365, 100)
    (W_z): Linear(in_features=200, out_features=100, bias=True)
    (U_r): Linear(in_features=100, out_features=100, bias=False)
    (W_r): Linear(in_features=100, out_features=100, bias=True)
    (W_h): Linear(in_features=200, out_features=100, bias=True)
    (W): Linear(in_features=128, out_features=100, bias=True)
    (U): Linear(in_features=128, out_features=100, bias=True)
    (U_i): Linear(in_features=200, out_features=100, bias=True)
    (W_o): 

## setting optimizer

In [4]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

optimizer = optim.Adam(enc_model.parameters(), lr=1e-3)
#optimizer = optim.SGD(enc_model.parameters(),lr=5e-2,momentum=0.9)
scheduler = lr_scheduler.ExponentialLR(optimizer, 0.9)
scheduler.step()

In [5]:
from MS_PredictModel import ms_peak_encoder,MS_Dataset
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np

pbar = None
train_dataset.batch_size = 10
vali_dataset.batch_size = 10

anneal_iter = 7400

def training(max_epoch = 100):
    global pbar
    total_step = 0
    meters = np.zeros(3)
    vali_meters = np.zeros(3)
    with open("log2.csv","w") as f:
        f.write("epoch,iter.,word,topo,assm,vali word,vali topo,vali assm\n")
    for epoch in range(max_epoch):
        print("epoch : ",epoch)
        for batch in train_dataset:
            x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
            total_step+=1
            #pbar.update(1)
            x = x.to('cuda')
            y = y.to('cuda')
            
            enc_model.zero_grad()
            dec_model.zero_grad()
            optimizer.zero_grad()
            
            h = enc_model(x,y)
            tree_vec = h[:,:h.shape[1]/2]
            mol_vec  = h[:,h.shape[1]/2:]
            _, x_tree_mess = dec_model.jtnn(*x_jtenc_holder)
            word_loss, topo_loss, word_acc, topo_acc = dec_model.decoder(x_batch,tree_vec)
            assm_loss, assm_acc = dec_model.assm(x_batch, x_jtmpn_holder, mol_vec , x_tree_mess)
            total_loss = word_loss+topo_loss+assm_loss
            total_loss.backward()
            optimizer.step()
            
            meters = meters + np.array([word_acc * 100, topo_acc * 100, assm_acc * 100])
            if total_step % 200 == 0:
                vali_total = 0
                for batch in vali_dataset:
                    x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
                    x = x.to('cuda')
                    y = y.to('cuda')
                    with torch.no_grad():
                        h = enc_model(x,y,False)
                        tree_vec = h[:,:h.shape[1]/2]
                        mol_vec  = h[:,h.shape[1]/2:]
                        _, x_tree_mess = dec_model.jtnn(*x_jtenc_holder)
                        word_loss, topo_loss, word_acc, topo_acc = dec_model.decoder(x_batch,tree_vec)
                        assm_loss, assm_acc = dec_model.assm(x_batch, x_jtmpn_holder, mol_vec , x_tree_mess)
                        vali_meters = vali_meters + np.array([word_acc * 100, topo_acc * 100, assm_acc * 100])
                        vali_total += 1
                meters /= 200
                vali_meters /= vali_total
                print "[%d] , Word: %.2f, Topo: %.2f, Assm: %.2f vali_Word: %.2f, vali_Topo: %.2f, vali_assm: %.2f, learning rate: %.4f" % \
                    (total_step, meters[0], meters[1], meters[2],vali_meters[0],vali_meters[1],vali_meters[2],scheduler.get_lr()[0])
                with open("log2.csv","a") as f:
                    f.write("%d,%d,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n" % (epoch,total_step,meters[0], meters[1], meters[2],vali_meters[0],vali_meters[1],vali_meters[2]))
                sys.stdout.flush()
                meters *= 0
                vali_meters *= 0
            if total_step % 200 == 0:
                torch.save(enc_model.state_dict(), "./enc_model" + "/model.iter-" + str(total_step))
            if total_step % anneal_iter == 0:
                scheduler.step()

#import pdb; pdb.set_trace()
try:
    #if pbar is None:
        #pbar = tqdm()
    training(100)
except RuntimeError as e:
    #if pbar is not None:
        #del pbar
    import traceback
    print(traceback.format_exc())
    #import pdb; pdb.set_trace()
    print(e)



('epoch : ', 0)


/usr/local/anaconda-aisiars/envs/jtvae/lib/python2.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/anaconda-aisiars/envs/jtvae/lib/python2.7/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[200] , Word: 46.34, Topo: 88.56, Assm: 89.66 vali_Word: 44.18, vali_Topo: 87.50, vali_assm: 89.72, learning rate: 0.0010
[400] , Word: 47.06, Topo: 87.92, Assm: 89.85 vali_Word: 51.79, vali_Topo: 87.45, vali_assm: 91.65, learning rate: 0.0010
[600] , Word: 47.33, Topo: 88.18, Assm: 89.43 vali_Word: 50.01, vali_Topo: 87.73, vali_assm: 89.48, learning rate: 0.0010
('epoch : ', 1)
[800] , Word: 47.71, Topo: 88.29, Assm: 90.67 vali_Word: 46.72, vali_Topo: 87.61, vali_assm: 89.34, learning rate: 0.0010
[1000] , Word: 46.89, Topo: 87.98, Assm: 89.47 vali_Word: 43.46, vali_Topo: 87.50, vali_assm: 91.01, learning rate: 0.0010
[1200] , Word: 47.97, Topo: 88.35, Assm: 89.90 vali_Word: 50.76, vali_Topo: 87.89, vali_assm: 90.66, learning rate: 0.0010
('epoch : ', 2)
[1400] , Word: 48.89, Topo: 88.62, Assm: 90.41 vali_Word: 48.27, vali_Topo: 88.96, vali_assm: 90.54, learning rate: 0.0010
[1600] , Word: 48.37, Topo: 88.82, Assm: 89.49 vali_Word: 47.70, vali_Topo: 88.36, vali_assm: 90.52, learning r

KeyboardInterrupt: 

In [9]:
def evaluation():
    with torch.no_grad():
        with open("evaluation.csv","w") as f:
            f.write("true,predict\n")
        for batch in vali_dataset:
            x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
            x = x.to('cuda')
            y = y.to('cuda')
            
            h = enc_model(x,y)
            tree_vec = h[:,:h.shape[1]/2]
            mol_vec  = h[:,h.shape[1]/2:]
            for num in range(h.size()[0]):
                with open("evaluation.csv","a") as f:
                    f.write(x_batch[num].smiles+","+dec_model.decode(tree_vec[num].view(1,latent_size/2),mol_vec[num].view(1,latent_size/2),False)+"\n")
            

evaluation()

In [10]:
import os
from rdkit import Chem
from rdkit.Chem import Draw

def _re_smiles(smiles1,smiles2):
    #print(smiles1,smiles2)
    smiles1 = Chem.MolToSmiles(Chem.MolFromSmiles(smiles1),True)
    smiles2 = Chem.MolToSmiles(Chem.MolFromSmiles(smiles2),True)
    return smiles1 == smiles2

def is_structural_isomer(smiles1,smiles2):
    def Molecular_formula(smiles):
        atoms = {}
        mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
        for atom in mol.GetAtoms():
            if not atom.GetSymbol() in atoms:
                atoms[atom.GetSymbol()] = 1
            else:
                atoms[atom.GetSymbol()] += 1
        return atoms
    atoms1 = Molecular_formula(smiles1)
    atoms2 = Molecular_formula(smiles2)
    return atoms1 == atoms2
    
def analyze_result(list_name="evaluation.csv",path="image_list"):
    if not os.path.exists(path):
        os.mkdir(path)
    with open(list_name,"r") as f:
        tp_list = [one.strip().split(",") for one in f.readlines()]
    
    print "Number of data: %d"% (len(tp_list)-1)
        
    true_list = [[i,one[0]] for i,one in enumerate(tp_list[1:]) if _re_smiles(one[0],one[1])]
    print "Number of matching: %d" % (len(true_list))
    print(true_list)
    
    true_list = [[i,one[0]] for i,one in enumerate(tp_list[1:]) if is_structural_isomer(one[0],one[1])]
    print "Number of matching: %d" % (len(true_list))
    print(true_list)
        
        
    for i,smiles in enumerate(tp_list[1:]):
        true_mol = Chem.MolFromSmiles(smiles[0])
        pred_mol = Chem.MolFromSmiles(smiles[1])
        image = Draw.MolsToImage([true_mol,pred_mol])
        image.save(os.path.join(path,"%d.png" % i))
    
analyze_result()

Number of data: 740
Number of matching: 23
[[11, 'CCCCCCCCCCCCC(CC(O)=O)NCc(c1)cccc1'], [13, 'O=C(CCCCC(=O)O[Si](C)(C)C)O[Si](C)(C)C'], [36, 'OCc(c1)cccc1'], [54, 'CCCCCCCC=CCC[Si](C)(C)C'], [67, 'CI'], [68, 'CCOCCOCC'], [79, 'CC(C)OC(C)C'], [113, 'COc(c1)ccc(OC)c1'], [192, 'CCCCCCCC(=O)OCCCC'], [258, 'CCOc(c1)c(ccc1)C(=O)O[Si](C)(C)C'], [306, 'C(C1)CCC(C1)C(=O)O[Si](C)(C)C'], [338, 'CC(C)COC(=O)c(c1)cccc1'], [376, 'C[Si](C)(C)OC(=O)c(c1)c(O[Si](C)(C)C)cc(c2)c(ccc2)1'], [381, 'ClC(Cl)C(Cl)Cl'], [443, 'CCCCCCCCO'], [515, 'Cc(c1)ccc(c1)CC(=O)O[Si](C)(C)C'], [603, 'c(c2)ccc(c2)C(C(C1)CCC1)C(=O)O[Si](C)(C)C'], [621, 'CCCCCCC(=O)CC'], [639, 'CCCCCCCCCCCCC(=O)O[Si](C)(C)C'], [661, 'OCC(O)Cn(c2)c(C(=O)1)c(n2)N(C)C(=O)N(C)1'], [699, 'CCCCOc(c1)ccc(c1)C(=O)O[Si](C)(C)C'], [707, 'O=C(CC(C1)CCCC1)O[Si](C)(C)C'], [713, 'O=C(C1)C(C)C(=O)C1']]
Number of matching: 35
[[11, 'CCCCCCCCCCCCC(CC(O)=O)NCc(c1)cccc1'], [13, 'O=C(CCCCC(=O)O[Si](C)(C)C)O[Si](C)(C)C'], [36, 'OCc(c1)cccc1'], [41, 'CC(C)(C)c(c1)c